In [2]:
''' Libraries ''' 
import requests
import os
import pandas as pd
from fuzzywuzzy import process, fuzz
from urllib.request import urlopen

In [15]:
def tag(nombre, api):
    name = nombre + '_'+api
    return name
tabla = pd.read_excel(r'../Merge.xlsx')
#tabla.drop(columns='Unnamed: 0', inplace=True)
columnas_sharadar = ['ticker', 'reportperiod', 'datekey'] +  tabla.tag_sharadar.tolist()
for i in range(len(columnas_sharadar)):
    columnas_sharadar[i] = columnas_sharadar[i].lower()

tabla.head()

,tag_tiingo,desc_tiingo,tag_alpha,desc_alpha,tag_finmodprep,tag_finn,tag_eod,tag_sharadar
0,totalAssets,Total Assets,totalAssets,"Assets, Total",totalAssets,totalAssets,totalAssets,ASSETS
1,totalLiabilities,Total Liabilities,totalLiabilities,"Liabilities, Total",totalLiabilities,totalLiabilities,totalLiab,LIABILITIES
2,sga,"Selling, General & Administrative",sellingGeneralAndAdministrative,"Selling, General and Administrative Expense, T...",sellingGeneralAndAdministrativeExpenses,sgaExpense,sellingGeneralAdministrative,SGNA
3,rnd,Research & Development,researchAndDevelopment,"Research and Development Expense, Total",researchAndDevelopmentExpenses,researchDevelopment,researchDevelopment,RND
4,ebitda,EBITDA,ebitda,"Earnings Before Interest, Taxes, Depreciation,...",ebitda,ebit,ebitda,EBITDA


In [16]:
def sharadar(tick):
    import nasdaqdatalink
    nasdaqdatalink.ApiConfig.api_key = "rxxWsjphD1YghSYo-LWD"

    df_sharadar = nasdaqdatalink.get_table('SHARADAR/SF1', ticker=f'{tick}')    
    df_sharadar = df_sharadar[columnas_sharadar]
    ''' add _sharadar to column names'''
    dict_sharadar = {}
    for i in range(len(columnas_sharadar)):
        dict_sharadar[columnas_sharadar[i]] = tag(columnas_sharadar[i], 'sharadar')
    df_sharadar.rename(columns=dict_sharadar, inplace=True)
    ''' datekey to date_time type''' 
    df_sharadar.datekey_sharadar = pd.to_datetime(df_sharadar.datekey_sharadar)
    df_sharadar.datekey_sharadar = df_sharadar.datekey_sharadar.dt.strftime('%Y-%m')
    return df_sharadar

In [17]:
df_sharadar = sharadar('AAPL')
df_sharadar

,ticker_sharadar,reportperiod_sharadar,datekey_sharadar,assets_sharadar,liabilities_sharadar,sgna_sharadar,rnd_sharadar,ebitda_sharadar,deferredrev_sharadar,cor_sharadar,...,inventory_sharadar,opinc_sharadar,accoci_sharadar,payables_sharadar,cashneq_sharadar,dps_sharadar,receivables_sharadar,capex_sharadar,equity_sharadar,opinc_sharadar
None,,,,,,,,,,,,,,,,,,,,,
0,AAPL,2022-09-24,2022-09,3.527550e+11,3.020830e+11,2.509400e+10,2.625100e+10,1.331380e+11,7.912000e+09,2.235460e+11,...,4.946000e+09,1.194370e+11,-1.110900e+10,6.411500e+10,2.364600e+10,0.900,6.093200e+10,-1.070800e+10,5.067200e+10,1.194370e+11
1,AAPL,2021-09-25,2021-09,3.510020e+11,2.879120e+11,2.197300e+10,2.191400e+10,1.231360e+11,7.612000e+09,2.129810e+11,...,6.580000e+09,1.089490e+11,1.630000e+08,5.476300e+10,3.494000e+10,0.850,5.150600e+10,-1.108500e+10,6.309000e+10,1.089490e+11
2,AAPL,2020-09-26,2020-09,3.238880e+11,2.585490e+11,1.991600e+10,1.875200e+10,8.102000e+10,6.643000e+09,1.695590e+11,...,4.061000e+09,6.628800e+10,-4.060000e+08,4.229600e+10,3.801600e+10,0.795,3.744500e+10,-7.309000e+09,6.533900e+10,6.628800e+10
3,AAPL,2019-09-28,2019-09,3.385160e+11,2.480280e+11,1.824500e+10,1.621700e+10,8.186000e+10,5.522000e+09,1.617820e+11,...,4.106000e+09,6.393000e+10,-5.840000e+08,4.623600e+10,4.884400e+10,0.750,4.580400e+10,-1.049500e+10,9.048800e+10,6.393000e+10
4,AAPL,2018-09-29,2018-09,3.657250e+11,2.585780e+11,1.670500e+10,1.423600e+10,8.704600e+10,5.966000e+09,1.637560e+11,...,3.956000e+09,7.089800e+10,-3.454000e+09,5.588800e+10,2.591300e+10,0.680,4.899500e+10,-1.331300e+10,1.071470e+11,7.089800e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,AAPL,1998-09-25,1998-12,4.289000e+09,2.647000e+09,2.350000e+08,7.300000e+07,1.370000e+08,0.000000e+00,1.139000e+09,...,7.800000e+07,1.090000e+08,0.000000e+00,7.190000e+08,1.481000e+09,0.000,9.550000e+08,-7.000000e+06,1.642000e+09,1.090000e+08
461,AAPL,1998-06-26,1998-08,4.041000e+09,2.555000e+09,2.160000e+08,7.600000e+07,1.410000e+08,0.000000e+00,1.042000e+09,...,1.290000e+08,6.100000e+07,0.000000e+00,5.730000e+08,1.203000e+09,0.000,9.150000e+08,1.700000e+07,1.486000e+09,6.100000e+07
462,AAPL,1998-03-27,1998-05,3.963000e+09,2.575000e+09,2.230000e+08,7.500000e+07,8.700000e+07,0.000000e+00,1.056000e+09,...,2.570000e+08,5.100000e+07,0.000000e+00,5.230000e+08,1.285000e+09,0.000,8.070000e+08,-5.000000e+06,1.388000e+09,5.100000e+07


# $$ Sharadar$$ 